In [1]:
!pip install openai



  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/81/b2/9d8939c9ef73e6a2d5cb3366ef3fabd728a8de4729210d9af785c0edc6ec/openai-1.55.0-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/e4/f5/f2b75d2fc6f1a260f340f0e7c6a060f4dd2961cc16884ed851b0d18da06a/anyio-4.6.2.post1-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/56/95/9377bcb415797e44274b51d46e3249eba641711cf3348050f76ee7b15ffc/httpx-0.27.2-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/packages/1d/0c/e7ac4ac5327eff2b93bcd70e74ca76a0d6b0a6d2fc27f40707ddabfe7739/jiter-0.7.

In [32]:
import torch
import transformers
from ast import literal_eval
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
import json
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from peft import AutoPeftModelForCausalLM, LoraConfig

pd.set_option('display.max_columns', None)

In [ ]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42) 

In [ ]:
# from openai import OpenAI
# client = OpenAI(api_key='')

In [35]:
# Load the train dataset
# TODO Train Data 경로 입력
dataset = pd.read_csv('train.csv') 

# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
df = pd.DataFrame(records)

In [88]:
set_seed(42) 
sample_df = df

In [89]:
sample_df

,id,paragraph,question,choices,answer,question_plus
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]",2,None
1,generation-for-nlp-426,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...",(가)에 대한 설명으로 옳지 않은 것은?,"[고려 문종 때에 남경(南京)으로 승격되었다., 종루(鐘樓), 이현, 칠패 등에서 ...",1,None
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"[이곳에 대장도감을 설치하여 재조대장경을 만들었다., 지눌이 이곳에서 수선사 결사운...",4,None
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"[살수에서 수의 군대를 물리쳤다 ., 김춘추 의 신라 왕위 계승을 지원하였다 ., ...",2,None
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"[사창제를 실시하였다 ., 대전회통을 편찬하였다 ., 비변사의 기능을 강화하였다 ....",3,None
...,...,...,...,...,...,...
2026,generation-for-nlp-2893,"“헐값에 팔리는 냉동 오렌지주스만 잔뜩 사가고, 쿠폰을 오려 모았으며, 구멍 난 스...",잭 맥도널드가 남긴 유산의 총액은 얼마인가?,"[1억8760만달러, 5000만달러, 2억달러, 1억달러, 3억달러]",1,None
2027,generation-for-nlp-2894,"넷기어코리아(지사장 김진겸, 이하 넷기어)가 뮤럴 캔버스를 가지고 넷기어 SNS에서...",넷기어가 모집하는 '뮤럴 공식 도슨트'의 주요 역할은 무엇인가?,"[전시물 설명 및 안내, 고객 서비스 제공, 제품 판매 촉진, 온라인 강좌 콘텐츠 ...",4,None
2028,generation-for-nlp-2895,서울 성동구 옥수동과 금호동은 맞붙어 있는 동네지만 아파트값은 같은 면적에서 1억원...,옥수동과 금호동의 아파트값 차이가 발생한 주된 이유는 무엇인가?,"[옥수동의 재개발이 먼저 이루어졌다, 금호동의 인구가 더 많다, 옥수동이 더 많은 ...",1,None
2029,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?,"[셀트리온, 삼성전자, LG화학, 한화그룹, 현대자동차]",1,None


In [90]:
# 새로운 'category' 열 추가
sample_df['category'] = ''

In [91]:
def classify_question(paragraph, question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "주어진 지문과 질문을 분석하여 해당 문제의 과목과 세부 주제를 분류해주세요."},
            {"role": "user", "content": f"""
        다음 지문과 질문을 읽고 아래 카테고리 중 가장 적합한 것을 선택하여 분류해주세요:

        지문: 로마 제국의 쇠락에 대한 설명
        질문: 저자가 로마 제국의 쇠락 원인으로 본 것은?
        분류 결과: 사회-세계사

        지문: 인도 총선에 대한 뉴스 기사
        질문: 이번 인도 총선의 주요 후보는 누구인가?
        분류 결과: 사회-정치와법

        지문: {paragraph}
        질문: {question}

        카테고리:
        1. 국어-독서
        2. 국어-문학
        3. 국어-화법과작문
        4. 국어-언어
        5. 국어-매체
        6. 사회-생활과윤리
        7. 사회-윤리와사상
        8. 사회-한국지리
        9. 사회-세계지리
        10. 사회-동아시아사
        11. 사회-세계사
        12. 사회-경제
        13. 사회-정치와법
        14. 사회-문화

        분류 결과:
        """}
        ],
        max_tokens=20
    )
    return response.choices[0].message.content.strip()

# 샘플 데이터에 대해 주제 분류 수행
for index, row in sample_df.iterrows():
    category = classify_question(row['paragraph'], row['question'])
    # '분류 결과:' 문구 제거 및 앞뒤 공백 제거
    category = category.replace('분류 결과:', '').strip()
    sample_df.at[index, 'category'] = category
    print(f"ID: {row['id']}")
    print(f"Paragraph: {row['paragraph'][:10]}...")  # 처음 100자만 출력
    print(f"Question: {row['question']}")
    print(f"Classified as: {category}\n")

ID: generation-for-nlp-425
Paragraph: 상소하여 아뢰기를 ...
Question: 상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?
Classified as: 사회-한국사

ID: generation-for-nlp-426
Paragraph: (가)은/는 의병계...
Question: (가)에 대한 설명으로 옳지 않은 것은?
Classified as: 사회-한국지리

ID: generation-for-nlp-427
Paragraph: 나는 삼한(三韓) ...
Question: (가) 지역에 대한 설명으로 옳은 것은?
Classified as: 사회-동아시아사

ID: generation-for-nlp-428
Paragraph: 이 날 소정방이 부...
Question: 밑줄 친 ‘그’에 대한 설명으로 옳은 것은?
Classified as: 사회-동아시아사

ID: generation-for-nlp-429
Paragraph: 선비들 수만 명이 ...
Question: (가) 인물이 추진한 정책으로 옳지 않은 것은?
Classified as: 사회-한국지리

ID: generation-for-nlp-431
Paragraph: (가)의 사신 고제...
Question: (가) 국가에 대한 설명으로 옳은 것은?
Classified as: 사회-동아시아사

ID: generation-for-nlp-432
Paragraph: (가)신라의 한강 ...
Question: 다음 사건을 시기 순으로 바르게 나열한 것은?
Classified as: 사회-동아시아사

ID: generation-for-nlp-433
Paragraph: 신돈이 (가)을/를...
Question: (가)에 대한 설명으로 옳은 것은?
Classified as: 사회-한국지리

ID: generation-for-nlp-434
Paragraph: (가) 황제가 영원...
Question: (가) 재위 기간에 있었던 사실이 아닌 것은?
Classified as: 국

In [92]:
# 분류 결과 출력
print(sample_df[['id', 'category']])

# 카테고리별 비율 계산
category_counts = sample_df['category'].value_counts()
category_percentages = category_counts / len(sample_df) * 100

print("\n카테고리별 비율:")
for category, percentage in category_percentages.items():
    print(f"{category}: {percentage:.2f}%")

                           id  category
0      generation-for-nlp-425    사회-한국사
1      generation-for-nlp-426   사회-한국지리
2      generation-for-nlp-427  사회-동아시아사
3      generation-for-nlp-428  사회-동아시아사
4      generation-for-nlp-429   사회-한국지리
...                       ...       ...
2026  generation-for-nlp-2893     사회-경제
2027  generation-for-nlp-2894     사회-문화
2028  generation-for-nlp-2895     사회-경제
2029  generation-for-nlp-2896     사회-경제
2030  generation-for-nlp-2899  사회-윤리와사상

[2031 rows x 2 columns]

카테고리별 비율:
사회-경제: 37.47%
사회-정치와법: 19.60%
사회-문화: 14.62%
사회-윤리와사상: 6.01%
사회-세계사: 4.38%
국어-문학: 2.56%
국어-독서: 1.72%
사회-한국지리: 1.58%
사회-동아시아사: 1.43%
사회-생활과윤리: 1.33%
12. 사회-경제: 1.28%
사회-교육: 1.13%
사회-심리학: 0.54%
국어-언어: 0.49%
2. 국어-문학: 0.39%
사회-환경: 0.39%
4. 국어-언어: 0.30%
6. 사회-생활과윤리: 0.25%
사회-한국사: 0.20%
과학-지구과학: 0.10%
7. 사회-윤리와사상: 0.10%
수학-통계: 0.10%
사회-노동과 경제: 0.10%
국어-화법과작문: 0.10%
과학-심리학: 0.10%
사회-사회문제: 0.10%
사회-의학: 0.10%
14. 사회-문화: 0.10%
사회-세계지리: 0.10%
사회-보건: 0.10%
심리-심리학: 0.10%
사회-교통과인프라: 0.05%
자연-과

In [93]:
sample_df

,id,paragraph,question,choices,answer,question_plus,category
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]",2,None,사회-한국사
1,generation-for-nlp-426,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...",(가)에 대한 설명으로 옳지 않은 것은?,"[고려 문종 때에 남경(南京)으로 승격되었다., 종루(鐘樓), 이현, 칠패 등에서 ...",1,None,사회-한국지리
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"[이곳에 대장도감을 설치하여 재조대장경을 만들었다., 지눌이 이곳에서 수선사 결사운...",4,None,사회-동아시아사
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"[살수에서 수의 군대를 물리쳤다 ., 김춘추 의 신라 왕위 계승을 지원하였다 ., ...",2,None,사회-동아시아사
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"[사창제를 실시하였다 ., 대전회통을 편찬하였다 ., 비변사의 기능을 강화하였다 ....",3,None,사회-한국지리
...,...,...,...,...,...,...,...
2026,generation-for-nlp-2893,"“헐값에 팔리는 냉동 오렌지주스만 잔뜩 사가고, 쿠폰을 오려 모았으며, 구멍 난 스...",잭 맥도널드가 남긴 유산의 총액은 얼마인가?,"[1억8760만달러, 5000만달러, 2억달러, 1억달러, 3억달러]",1,None,사회-경제
2027,generation-for-nlp-2894,"넷기어코리아(지사장 김진겸, 이하 넷기어)가 뮤럴 캔버스를 가지고 넷기어 SNS에서...",넷기어가 모집하는 '뮤럴 공식 도슨트'의 주요 역할은 무엇인가?,"[전시물 설명 및 안내, 고객 서비스 제공, 제품 판매 촉진, 온라인 강좌 콘텐츠 ...",4,None,사회-문화
2028,generation-for-nlp-2895,서울 성동구 옥수동과 금호동은 맞붙어 있는 동네지만 아파트값은 같은 면적에서 1억원...,옥수동과 금호동의 아파트값 차이가 발생한 주된 이유는 무엇인가?,"[옥수동의 재개발이 먼저 이루어졌다, 금호동의 인구가 더 많다, 옥수동이 더 많은 ...",1,None,사회-경제
2029,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?,"[셀트리온, 삼성전자, LG화학, 한화그룹, 현대자동차]",1,None,사회-경제


In [106]:
sample_df12 = sample_df.copy()

In [107]:
sample_df12

,id,paragraph,question,choices,answer,question_plus,category
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]",2,None,사회-한국사
1,generation-for-nlp-426,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...",(가)에 대한 설명으로 옳지 않은 것은?,"[고려 문종 때에 남경(南京)으로 승격되었다., 종루(鐘樓), 이현, 칠패 등에서 ...",1,None,사회-한국지리
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"[이곳에 대장도감을 설치하여 재조대장경을 만들었다., 지눌이 이곳에서 수선사 결사운...",4,None,사회-동아시아사
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"[살수에서 수의 군대를 물리쳤다 ., 김춘추 의 신라 왕위 계승을 지원하였다 ., ...",2,None,사회-동아시아사
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"[사창제를 실시하였다 ., 대전회통을 편찬하였다 ., 비변사의 기능을 강화하였다 ....",3,None,사회-한국지리
...,...,...,...,...,...,...,...
2026,generation-for-nlp-2893,"“헐값에 팔리는 냉동 오렌지주스만 잔뜩 사가고, 쿠폰을 오려 모았으며, 구멍 난 스...",잭 맥도널드가 남긴 유산의 총액은 얼마인가?,"[1억8760만달러, 5000만달러, 2억달러, 1억달러, 3억달러]",1,None,사회-경제
2027,generation-for-nlp-2894,"넷기어코리아(지사장 김진겸, 이하 넷기어)가 뮤럴 캔버스를 가지고 넷기어 SNS에서...",넷기어가 모집하는 '뮤럴 공식 도슨트'의 주요 역할은 무엇인가?,"[전시물 설명 및 안내, 고객 서비스 제공, 제품 판매 촉진, 온라인 강좌 콘텐츠 ...",4,None,사회-문화
2028,generation-for-nlp-2895,서울 성동구 옥수동과 금호동은 맞붙어 있는 동네지만 아파트값은 같은 면적에서 1억원...,옥수동과 금호동의 아파트값 차이가 발생한 주된 이유는 무엇인가?,"[옥수동의 재개발이 먼저 이루어졌다, 금호동의 인구가 더 많다, 옥수동이 더 많은 ...",1,None,사회-경제
2029,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?,"[셀트리온, 삼성전자, LG화학, 한화그룹, 현대자동차]",1,None,사회-경제


In [105]:
unique_categories = sample_df12['category'].unique()
unique_categories

array(['사회-한국사', '사회-한국지리', '사회-동아시아사', '국어-문학', '3. 국어-화법과작문',
       '사회-윤리와사상', '사회-정치와법', '사회의 역사적 사건에 대한 질문이므로, 해당 지문과 질문은 "사회-',
       '사회-문화', '국어-독서', '14. 사회-문화', '2. 국어-문학', '사회-세계사', '사회-경제',
       '사회-자연과학', '사회-생활과윤리', '사회-유럽사', '과학-천문학', '사회-인구와사회', '사회-세계지리',
       '12. 사회-경제', '11. 사회-경제', '6. 사회-생활과윤리',
       '지문: 멜라니는 매일 점심값으로 개당 1달러', '심리-심리학', '사회-교육', '사회-연구방법론',
       '사회-심리학', '6. 수학-통계', '과학-신경과학', '과학-생물', '수학-통계', '국어-언어',
       '4. 국어-언어', '5. 국어-매체', '7. 사회-윤리와사상', '과목: 과학\n세부 주제: 심리학-수면 연구',
       '국어-화법과작문', '과학-심리학', '사회-과학', '사회-정신건강', '자연-생물학', '과학-생명과학',
       '과학-생물학', '8. 사회-정치와법', '과목: 사회  \n세부 주제: 사회-문화', '결과: 사회-윤리와사상',
       '사회-노동과 경제', '사회-역사', '1. 국어-독서',
       '주어진 지문과 질문을 분석한 결과, 해당 문제는 다음과 같이 분류됩니다', '13. 사회-정치와법', '사회-환경',
       '사회-주택 정책', '과목: 과학  \n세부 주제: 생명과학-신경과학', '사회-환경과기후변화', '국어-매체',
       '사회-주거와부동산', '사회-사건과재난', '사회-스포츠', '과학-기술', '사회-사회문제', '사회-보건과안전',
       '사회의 생활과 윤리와 관련된 주제이므로 다음과 같이 분류합니다.', '사회-의학', '과학-기상학',

In [108]:
import re

def remove_numbers_and_dots(text):
    return re.sub(r'^\d+\.\s*', '', text)

# category 컬럼에 함수 적용
sample_df12['category_cleaned'] = sample_df12['category'].apply(remove_numbers_and_dots)

# 결과 확인
print(sample_df12['category_cleaned'].unique())

['사회-한국사' '사회-한국지리' '사회-동아시아사' '국어-문학' '국어-화법과작문' '사회-윤리와사상' '사회-정치와법'
 '사회의 역사적 사건에 대한 질문이므로, 해당 지문과 질문은 "사회-' '사회-문화' '국어-독서' '사회-세계사' '사회-경제'
 '사회-자연과학' '사회-생활과윤리' '사회-유럽사' '과학-천문학' '사회-인구와사회' '사회-세계지리'
 '지문: 멜라니는 매일 점심값으로 개당 1달러' '심리-심리학' '사회-교육' '사회-연구방법론' '사회-심리학' '수학-통계'
 '과학-신경과학' '과학-생물' '국어-언어' '국어-매체' '과목: 과학\n세부 주제: 심리학-수면 연구' '과학-심리학'
 '사회-과학' '사회-정신건강' '자연-생물학' '과학-생명과학' '과학-생물학' '과목: 사회  \n세부 주제: 사회-문화'
 '결과: 사회-윤리와사상' '사회-노동과 경제' '사회-역사'
 '주어진 지문과 질문을 분석한 결과, 해당 문제는 다음과 같이 분류됩니다' '사회-환경' '사회-주택 정책'
 '과목: 과학  \n세부 주제: 생명과학-신경과학' '사회-환경과기후변화' '사회-주거와부동산' '사회-사건과재난' '사회-스포츠'
 '과학-기술' '사회-사회문제' '사회-보건과안전' '사회의 생활과 윤리와 관련된 주제이므로 다음과 같이 분류합니다.'
 '사회-의학' '과학-기상학' '사회-주거와도시개발' '과목: 국어  \n세부 주제: 국어-독서' '자연-생명과학' '자연-과학'
 '사회-교통과인프라' '사회-기술과사회' '과학-기후 및 환경 과학' '과학-화학' '사회-보건'
 '과목: 과학  \n세부 주제: 물리학/재료과학' '결과: 사회-경제' '사회-교통과환경' '사회-범죄와사회' '사회-사고와재난'
 '과학-지구과학' '과목: 과학  \n세부 주제: 환경과학' '사회-정보 및 사이버 보안' '사회-건강과환경'
 '사회-사고 및 재난 관리' '사회-의료와학' '사회-환경과 지속가능성'
 '지문: 서울시 산하 SH공사는 오는 18일부터 

In [109]:
sample_df12

,id,paragraph,question,choices,answer,question_plus,category,category_cleaned
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]",2,None,사회-한국사,사회-한국사
1,generation-for-nlp-426,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...",(가)에 대한 설명으로 옳지 않은 것은?,"[고려 문종 때에 남경(南京)으로 승격되었다., 종루(鐘樓), 이현, 칠패 등에서 ...",1,None,사회-한국지리,사회-한국지리
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"[이곳에 대장도감을 설치하여 재조대장경을 만들었다., 지눌이 이곳에서 수선사 결사운...",4,None,사회-동아시아사,사회-동아시아사
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"[살수에서 수의 군대를 물리쳤다 ., 김춘추 의 신라 왕위 계승을 지원하였다 ., ...",2,None,사회-동아시아사,사회-동아시아사
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"[사창제를 실시하였다 ., 대전회통을 편찬하였다 ., 비변사의 기능을 강화하였다 ....",3,None,사회-한국지리,사회-한국지리
...,...,...,...,...,...,...,...,...
2026,generation-for-nlp-2893,"“헐값에 팔리는 냉동 오렌지주스만 잔뜩 사가고, 쿠폰을 오려 모았으며, 구멍 난 스...",잭 맥도널드가 남긴 유산의 총액은 얼마인가?,"[1억8760만달러, 5000만달러, 2억달러, 1억달러, 3억달러]",1,None,사회-경제,사회-경제
2027,generation-for-nlp-2894,"넷기어코리아(지사장 김진겸, 이하 넷기어)가 뮤럴 캔버스를 가지고 넷기어 SNS에서...",넷기어가 모집하는 '뮤럴 공식 도슨트'의 주요 역할은 무엇인가?,"[전시물 설명 및 안내, 고객 서비스 제공, 제품 판매 촉진, 온라인 강좌 콘텐츠 ...",4,None,사회-문화,사회-문화
2028,generation-for-nlp-2895,서울 성동구 옥수동과 금호동은 맞붙어 있는 동네지만 아파트값은 같은 면적에서 1억원...,옥수동과 금호동의 아파트값 차이가 발생한 주된 이유는 무엇인가?,"[옥수동의 재개발이 먼저 이루어졌다, 금호동의 인구가 더 많다, 옥수동이 더 많은 ...",1,None,사회-경제,사회-경제
2029,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?,"[셀트리온, 삼성전자, LG화학, 한화그룹, 현대자동차]",1,None,사회-경제,사회-경제


In [112]:
# 지정된 카테고리 목록
valid_categories = [
    '국어-독서', '국어-문학', '국어-화법과작문', '국어-언어', '국어-매체',
    '사회-생활과윤리', '사회-윤리와사상', '사회-한국지리', '사회-세계지리',
    '사회-동아시아사', '사회-세계사', '사회-경제', '사회-정치와법', '사회-문화'
]

# 카테고리 열에서 숫자와 점 제거 (예: '1.', '2.' 등)
sample_df12['cleaned_category'] = sample_df12['category'].str.replace(r'^\d+\.\s*', '', regex=True)

# 지정된 카테고리에 해당하지 않는 데이터 추출
non_standard_categories = sample_df12[~sample_df12['cleaned_category'].isin(valid_categories)]


In [113]:
non_standard_categories

,id,paragraph,question,choices,answer,question_plus,category,category_cleaned,cleaned_category
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]",2,None,사회-한국사,사회-한국사,사회-한국사
23,generation-for-nlp-452,"풍토에 따라 곡식을 심고 가꾸는 법이 다르니, 고을의 경험 많은 농부를 각 도의 감...",밑줄 친 ‘왕’의 업적으로 옳은 것은?,"[공법을 제정하였다 ., 한양으로 도읍을 옮겼다 ., 경국대 전 을 완성하였다 .,...",1,None,"사회의 역사적 사건에 대한 질문이므로, 해당 지문과 질문은 ""사회-","사회의 역사적 사건에 대한 질문이므로, 해당 지문과 질문은 ""사회-","사회의 역사적 사건에 대한 질문이므로, 해당 지문과 질문은 ""사회-"
34,generation-for-nlp-465,(가) 김시민이 진주성에서 일본군을 저지하였다.(나) 조선수군이 명량해전에서 크게 ...,임진왜란의 주요 사건을 시기 순으로 바르게 나열한 것은?,"[(가)→(다)→(마)→(라)→(나), (가)→(마)→(다)→(나)→(라), (다)...",4,None,사회-한국사,사회-한국사,사회-한국사
54,generation-for-nlp-491,"왕이 복주에 이르렀다.정세운은 성품이 충성스럽고 청렴하였는데, 왕의 파천(播遷) 이...",밑줄 친 ‘왕’의 재위 기간에 있었던 일로 옳은 것은?,"[이제현에 의해 사략이 편찬되었다., 직지 심체요절이 금속활자로 인쇄되었다., 정치...",1,None,사회-한국사,사회-한국사,사회-한국사
86,generation-for-nlp-529,먼저 우주가 구형이라는 점에 주목해야 한다. 부분적으로는 연결점이 필요 없는 완전한...,위 글을 보았을 때 구체 형태의 우주에 관한 코페르니쿠스의 주장은 무엇을 근거로 하...,"[관찰과 귀납, 과거 기록의 권위, 실험, 제1원칙으로부터 추론]",1,None,사회-자연과학,사회-자연과학,사회-자연과학
...,...,...,...,...,...,...,...,...,...
1988,generation-for-nlp-2849,보이스피싱 사기 현행범이 서울의 한 새마을금고에서 범죄 수익금을 인출하려다 새마을금...,보이스피싱 사기 현행범이 체포된 장소는 어디인가?,"[서울 사당동 동작새마을금고, 서울 강남구 하나은행, 서울 종로구 국민은행, 서울 ...",1,None,사회-범죄와 법,사회-범죄와 법,사회-범죄와 법
1990,generation-for-nlp-2851,철도부지 등에 건설되는 공공임대주택인 ‘행복주택’이 주거·호텔·상업·업무시설 등이 ...,행복주택의 주거환경 개선을 통한 주요 목적은 무엇인가?,"[슬럼화 방지와 일자리 창출, 주거비 절감과 세금 인하, 주택 공급 증가와 외국인 ...",1,None,사회-주택과사회복지,사회-주택과사회복지,사회-주택과사회복지
1996,generation-for-nlp-2857,미국에 이어 스페인에서도 에볼라 바이러스에 감염된 환자가 확인됐다.월스트리트저널(W...,스페인에서 에볼라 바이러스에 감염된 간호사가 치료한 환자의 이름은 무엇인가?,"[마누엘 가르시아 비에호, 토머스 에릭 던컨, 시에라리온, 월스트리트저널, 스페인 ...",1,None,사회-생명과학,사회-생명과학,사회-생명과학
2012,generation-for-nlp-2875,롯데건설이 서울 금천구 독산동에 건설 중인 ‘롯데캐슬 골드파크’는 도심에 들어서는 ...,롯데캐슬 골드파크의 주요 특징 중 하나로 옳지 않은 것은 무엇인가?,"[주거·상업·호텔·문화·공공·교육시설이 모두 갖춰진다., 단지 안에 대형 공원이 조...",4,None,카테고리: 사회-경제,카테고리: 사회-경제,카테고리: 사회-경제


In [114]:
unique_categories = non_standard_categories['cleaned_category'].unique()
print(unique_categories)

['사회-한국사' '사회의 역사적 사건에 대한 질문이므로, 해당 지문과 질문은 "사회-' '사회-자연과학' '사회-유럽사'
 '과학-천문학' '사회-인구와사회' '지문: 멜라니는 매일 점심값으로 개당 1달러' '심리-심리학' '사회-교육'
 '사회-연구방법론' '사회-심리학' '수학-통계' '과학-신경과학' '과학-생물' '과목: 과학\n세부 주제: 심리학-수면 연구'
 '과학-심리학' '사회-과학' '사회-정신건강' '자연-생물학' '과학-생명과학' '과학-생물학'
 '과목: 사회  \n세부 주제: 사회-문화' '결과: 사회-윤리와사상' '사회-노동과 경제' '사회-역사'
 '주어진 지문과 질문을 분석한 결과, 해당 문제는 다음과 같이 분류됩니다' '사회-환경' '사회-주택 정책'
 '과목: 과학  \n세부 주제: 생명과학-신경과학' '사회-환경과기후변화' '사회-주거와부동산' '사회-사건과재난' '사회-스포츠'
 '과학-기술' '사회-사회문제' '사회-보건과안전' '사회의 생활과 윤리와 관련된 주제이므로 다음과 같이 분류합니다.'
 '사회-의학' '과학-기상학' '사회-주거와도시개발' '과목: 국어  \n세부 주제: 국어-독서' '자연-생명과학' '자연-과학'
 '사회-교통과인프라' '사회-기술과사회' '과학-기후 및 환경 과학' '과학-화학' '사회-보건'
 '과목: 과학  \n세부 주제: 물리학/재료과학' '결과: 사회-경제' '사회-교통과환경' '사회-범죄와사회' '사회-사고와재난'
 '과학-지구과학' '과목: 과학  \n세부 주제: 환경과학' '사회-정보 및 사이버 보안' '사회-건강과환경'
 '사회-사고 및 재난 관리' '사회-의료와학' '사회-환경과 지속가능성'
 '지문: 서울시 산하 SH공사는 오는 18일부터 장기전세주택' '지문: 터미널전자상가로 사용됐던 서울 용산관광버스터'
 '사회-범죄와법' '사회-범죄와 법' '사회-주택과사회복지' '사회-생명과학' '카테고리: 사회-경제'
 '과목: 과학\n세부 주제: 생명과학']
